In [3]:
# Importing Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator

In [4]:
# Setting the Path
path = 'uv_images'

In [5]:
# Loading the images
train_data_dir = os.path.join(path, 'Train')
test_data_dir = os.path.join(path, 'Test')

In [6]:
# Image Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
# Training the Model
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
    )

Found 71 images belonging to 2 classes.
Found 71 images belonging to 2 classes.


In [8]:
model = Sequential()

#Adding convolutional layers
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Adding a second convolutional layer
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [9]:
# Compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
# # Training the Model

# Assuming train_generator and test_generator are instances of ImageDataGenerator
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
# Calculate the number of steps per epoch and validation steps based on the dataset size
train_steps_per_epoch = len(train_generator)
validation_steps = len(test_generator)

# Reduce the number of epochs
epochs = 20  # You can adjust this based on your observations

model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping])


Epoch 1/20
3/3 [==============================] - 8s 3s/step - loss: 0.9212 - accuracy: 0.4930 - val_loss: 0.6513 - val_accuracy: 0.5775
Epoch 2/20
3/3 [==============================] - 2s 555ms/step - loss: 0.7492 - accuracy: 0.5352 - val_loss: 0.6019 - val_accuracy: 0.6761
Epoch 3/20
3/3 [==============================] - 3s 1s/step - loss: 0.5975 - accuracy: 0.7042 - val_loss: 0.5510 - val_accuracy: 0.9014
Epoch 4/20
3/3 [==============================] - 3s 1s/step - loss: 0.5339 - accuracy: 0.8310 - val_loss: 0.5111 - val_accuracy: 0.7606
Epoch 5/20
3/3 [==============================] - 3s 1s/step - loss: 0.5024 - accuracy: 0.7183 - val_loss: 0.4068 - val_accuracy: 0.8732
Epoch 6/20
3/3 [==============================] - 3s 1s/step - loss: 0.3987 - accuracy: 0.8873 - val_loss: 0.2854 - val_accuracy: 0.9437
Epoch 7/20
3/3 [==============================] - 3s 1s/step - loss: 0.2968 - accuracy: 0.9577 - val_loss: 0.2199 - val_accuracy: 0.9296
Epoch 8/20
3/3 [======================

In [15]:
# Saving the Model
model.save('kidney_stone_detection_model2.h5')

c:\Users\Sumukh's\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
from keras.models import load_model
model = load_model('kidney_stone_detection_model2.h5')

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                   

In [18]:
from keras.preprocessing.image import ImageDataGenerator

# Assuming you have a test dataset stored in a directory called 'test_data'
test_data_directory = 'C:\\Users\\Sumukh\'s\\Desktop\\BE_Project\\Kidney-Stone-Detection-main\\uv_images\\Test'

# Create a test data generator
test_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Or any other preprocessing steps

# Flow test data from the directory using the test data generator
test_generator = test_datagen.flow_from_directory(
    test_data_directory,
    target_size=(150, 150),  # Specify the target size of your input images
    batch_size=32,  # Adjust batch size according to your requirements
    class_mode='binary',  # Set class_mode to 'binary' or 'categorical' depending on your problem
    shuffle=False  # Set shuffle to False if you want to evaluate in order
)

# Use the test generator in the model.evaluate() function
model.evaluate(test_generator, steps=1)


Found 71 images belonging to 2 classes.


1/1 [==============================] - 1s 720ms/step - loss: 0.0098 - accuracy: 1.0000


[0.009769842028617859, 1.0]

# SVM

In [31]:
# Importing the required Modules
import cv2
import joblib
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# ... (previous code for reading images and defining functions)

# Combining the features for both classes
features = feature_list_normal + feature_list_stone

# Standardize the features
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# Splitting the data into train and valid sets
X_train, X_valid, y_train, y_valid = train_test_split(features_standardized, labels, test_size=0.2, random_state=0)

# Define the parameter grid for GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': ['auto', 'scale', 0.01, 0.1, 1, 10],
              'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}

# Create the SVM model with custom kernel
def custom_kernel(img1, img2):
    # Implement your custom kernel function here
    # You might consider preprocessing ultrasound images differently
    return np.dot(img1.flatten(), img2.flatten())

svm_model = SVC(kernel=custom_kernel)

# Create GridSearchCV
grid_search = GridSearchCV(svm_model, param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best Parameters:", grid_search.best_params_)

# Predicting the Test Set with the best model
X_valid_standardized = scaler.transform(X_valid)
y_pred = grid_search.best_estimator_.predict(X_valid_standardized)

# Calculating the accuracy
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy after tuning:", accuracy)

# Save the model to a file
joblib.dump(grid_search.best_estimator_, 'svc_tuned.pkl')

# Load the tuned SVM model
svm_load = joblib.load("svc_tuned.pkl")


Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best Parameters: {'C': 1, 'gamma': 'auto', 'kernel': 'poly'}
Accuracy after tuning: 1.0


In [ ]:
# #Importing the required Modules
# import cv2 
# import numpy as np 
# from skimage.feature import hog 
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score 
# import matplotlib.pyplot as plt 
# from sklearn.model_selection import train_test_split 
# from sklearn.metrics import confusion_matrix
# import os

In [ ]:
#Defining a function to read images from the train and test folders
def read_images(path):
    images_list = []
    for filename in os.listdir(path):
        img = cv2.imread(os.path.join(path,filename))
        if img is not None:
            images_list.append(img)
    return images_list

In [ ]:
#Reading train images from the normal and stone folders
train_normal = read_images('C:\\Users\\Sumukh\'s\\Desktop\\BE_Project\\Kidney-Stone-Detection-main\\uv_images\\Train\\Normal')
train_stone = read_images('C:\\Users\\Sumukh\'s\\Desktop\\BE_Project\\Kidney-Stone-Detection-main\\uv_images\\Train\\Stones')



In [ ]:
#Creating a list of labels for training 
labels = ['Normal' for item in train_normal] + ['Stone' for item in train_stone]

In [ ]:
#Defining a function for HOG feature extraction
def extract_features(images):
    feature_list = []
    for img in images:
        fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16), 
                            cells_per_block=(1, 1), visualize=True, channel_axis=2)
        # Resize the HOG features to a fixed size
        fd = np.resize(fd, (2400, 1))
        # Flatten the array to 2 dimensions
        fd = fd.flatten()
        feature_list.append(fd)
    return feature_list

In [ ]:
#Extracting the HOG features from both normal and stone images
feature_list_normal = extract_features(train_normal)
feature_list_stone = extract_features(train_stone)

In [ ]:
print(len(feature_list_normal))
print(len(feature_list_stone))

39
32


In [ ]:
#Combining the features for both classes
features = feature_list_normal + feature_list_stone

In [ ]:
#Reading test images from the normal and stone folders
test_normal = read_images('C:\\Users\\Sumukh\'s\\Desktop\\BE_Project\\Kidney-Stone-Detection-main\\uv_images\\Test\\Normal')
test_stone = read_images('C:\\Users\\Sumukh\'s\\Desktop\\BE_Project\\Kidney-Stone-Detection-main\\uv_images\\Test\\Stones')

In [ ]:
#Creating a list of labels for testing 
test_labels = ['Normal' for item in test_normal] + ['Stone' for item in test_stone]

In [ ]:
#Creating a Feature Vector for Test Set
test_feature_list_normal = extract_features(test_normal)
test_feature_list_stone = extract_features(test_stone)

In [ ]:
print(len(test_feature_list_normal))
print(len(test_feature_list_stone))

39
32


In [ ]:
#Combining the features for both classes
test_features = test_feature_list_normal + test_feature_list_stone

In [ ]:
#Splitting the data into train and valid sets
X_train, X_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, random_state=0)

In [ ]:
# Print the shape of the first element in the X_train array
print(X_train[0].shape)

# Print the shape of the second element in the X_train array
print(X_train[1].shape)

# Print the shape of the last element in the X_train array
print(X_train[-1].shape)

(2400,)
(2400,)
(2400,)


In [ ]:
# Training a SVM Model
svc = SVC(kernel='rbf', C=1, gamma='auto')
svc.fit(X_train, y_train)

SVC(C=1, gamma='auto')

In [ ]:
# Predicting the Test Set
y_pred = svc.predict(X_valid)

In [ ]:
#Calculating the accuracy
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy : ", accuracy)

Accuracy :  0.6


In [ ]:
import sklearn.externals
import joblib

# Save the model to a file
joblib.dump(svc, 'svc1.pkl')

['svc1.pkl']

In [ ]:
import sklearn.externals
import joblib
svm_load = joblib.load("svc1.pkl") 

In [30]:
# # Creating the GUI
# from tkinter import *
# from PIL import Image, ImageTk
# from tkinter import filedialog
# from tkinter.filedialog import askopenfilename
# import keras.utils as image
# import numpy as np
# import cv2

# root = Tk()
# root.geometry('420x380')
# root.title('Kidney Stone Detection')

# # Function to Select Image
# def browse_btn():
#     global image_name
    
#     label_cnn.configure(text="")
#     label.configure(text="")
    
#     image_name = askopenfilename(title='Select Image')
#     img = Image.open(image_name)
#     img = img.resize((200, 200), Image.ANTIALIAS)
#     img = ImageTk.PhotoImage(img)
#     panel = Label(root, image=img)
#     panel.image = img
#     panel.grid(row=0, column=1, sticky='nw', padx=20, pady=28)

# # Function to Predict CNN
# # def predict_btn_cnn():
# #     global label_cnn
# #     global image_name
# #     test_img = image.load_img(image_name, target_size=(150, 150))
# #     test_img = image.img_to_array(test_img)
# #     test_img = np.expand_dims(test_img, axis=0)
# #     result = model.predict(test_img)
# #     if result[0][0] == 1:
# #         label_cnn.configure(text="Kidney Stone Detected")
# #     elif result[0][0] == 0:
# #         label_cnn.configure(text="No Kidney Stone Detected")
#     from keras.preprocessing import image

# # Function for Predict CNN
# def predict_btn_cnn():
#     global label_cnn
#     global image_name
    
#     # Load and preprocess the image
#     test_img = image.load_img(image_name, target_size=(150, 150))
#     test_img = image.img_to_array(test_img)
#     test_img = np.expand_dims(test_img, axis=0)
#     test_img = test_img / 255.0  # Normalize pixel values to the range [0, 1]

#     # Predict
#     result = model.predict(test_img)

#     # Print the raw prediction for debugging
#     print("Raw Prediction:", result)

#     # Displaying the output
#     if result[0][0] > 0.5:  # Assuming 0.5 as the threshold for binary classification
#         label_cnn.configure(text="Kidney Stone Detected")
#     else:
#         label_cnn.configure(text="No Kidney Stone Detected")


# # Function for Predict SVM
# # def predict_btn_svm():
# #     global label
# #     global image_name
# #     test_img = cv2.imread(image_name)
# #     # test_img = image.load_img(image_name, target_size=(150, 150))
# #     # test_img = image.img_to_array(test_img)
# #     feature_list_of_img = extract_features([test_img])
# #     result = svm_load.predict(feature_list_of_img)
# #     # Displaying the output
# #     if result[0] == 'Stone':
# #         label.configure(text="Kidney Stone Detected")
# #     elif result[0] == 'Normal':
# #         label.configure(text="No Kidney Stone Detected")
#         # Function for Predict SVM
# def predict_btn_svm():
#     global label
#     global image_name
#     test_img = cv2.imread(image_name)
    
#     # Extract features
#     feature_list_of_img = extract_features([test_img])
    
#     # Standardize features if needed
#     feature_list_of_img_standardized = scaler.transform(feature_list_of_img)
    
#     # Predict
#     result = svm_load.predict(feature_list_of_img_standardized)
    
#     # Print the raw prediction for debugging
#     print("Raw Prediction:", result)
    
#     # Displaying the output
#     if result[0] == 'Stone':
#         label.configure(text="Kidney Stone Detected")
#     elif result[0] == 'Normal':
#         label.configure(text="No Kidney Stone Detected")


# # Browse Button
# browsebtn = Button(master=root, text="Browse Image", command=browse_btn)
# browsebtn.grid(row=0, column=0, sticky='nw', padx=20, pady=20)

# # Predict Button CNN
# predictbtn = Button(master=root, text="Predict CNN", command=predict_btn_cnn)
# predictbtn.grid(row=1, column=0, sticky='nw', padx=20, pady=20)

# # Label Result CNN
# label_cnn = Label(root, text="")
# label_cnn.grid(row=1, column=1, sticky='nw', padx=20, pady=20)

# # Label Result SVM
# label = Label(root, text="")
# label.grid(row=2, column=1, sticky='nw', padx=20, pady=20)

# # Predict Button SVM
# predictbtnsvm = Button(master=root, text="Predict SVM", command=predict_btn_svm)
# predictbtnsvm.grid(row=2, column=0, sticky='nw', padx=20, pady=20)

# # Running the GUI
# root.mainloop()


C:\Users\Sumukh's\AppData\Local\Temp\ipykernel_14472\2327085567.py:23: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((200, 200), Image.ANTIALIAS)


1/1 [==============================] - 0s 32ms/step
Raw Prediction: [[0.00380866]]
Raw Prediction: ['Normal']


C:\Users\Sumukh's\AppData\Local\Temp\ipykernel_14472\2327085567.py:23: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((200, 200), Image.ANTIALIAS)


1/1 [==============================] - 0s 28ms/step
Raw Prediction: [[0.99984837]]
Raw Prediction: ['Normal']


C:\Users\Sumukh's\AppData\Local\Temp\ipykernel_14472\2327085567.py:23: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((200, 200), Image.ANTIALIAS)


...

C:\Users\Sumukh's\AppData\Local\Temp\ipykernel_14472\2327085567.py:23: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((200, 200), Image.ANTIALIAS)


1/1 [==============================] - 0s 30ms/step
Raw Prediction: [[0.00093561]]
Raw Prediction: ['Normal']


C:\Users\Sumukh's\AppData\Local\Temp\ipykernel_14472\2327085567.py:23: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((200, 200), Image.ANTIALIAS)


1/1 [==============================] - 0s 29ms/step
Raw Prediction: [[0.9998614]]
Raw Prediction: ['Stone']


Exception in Tkinter callback
Exception ignored in: <function PhotoImage.__del__ at 0x000001ED80272B00>
Traceback (most recent call last):
  File "c:\Users\Sumukh's\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\ImageTk.py", line 133, in __del__
    name = self.__photo.name
AttributeError: 'PhotoImage' object has no attribute '_PhotoImage__photo'
Traceback (most recent call last):
  File "c:\Users\Sumukh's\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py", line 3135, in open
    fp.seek(0)
AttributeError: 'str' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Sumukh's\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Sumukh's\AppData\Local\Temp\ipykernel_14472\2327085567.py", line 22, in browse_btn
    img = Image.open(image_name)
  File "c:\Users\Sumukh's\AppData\